In [ ]:
import pandas as pd

# Load your CSV
df = pd.read_csv("/content/jira_issues.csv")

# Show basic info
print("Shape:", df.shape)
print("\nColumns:\n", df.columns.tolist())

# Count of nulls and zeros per column
missing_summary = pd.DataFrame({
    "null_count": df.isnull().sum(),
    "zero_count": (df == 0).sum(),
    "non_null_count": df.notnull().sum(),
    "total_rows": len(df)
})
missing_summary["null_%"] = (missing_summary["null_count"] / len(df)) * 100
missing_summary["zero_%"] = (missing_summary["zero_count"] / len(df)) * 100

missing_summary = missing_summary.sort_values(by="null_%", ascending=False)
print("\n=== Missing/Zero Summary ===")
display(missing_summary.head(20))  # Show top 20 columns with most nulls


Shape: (63989, 18)

Columns:
 ['id', 'created', 'description', 'key', 'priority', 'project', 'project_name', 'repositoryname', 'resolution', 'resolved', 'status', 'title', 'type', 'updated', 'votes', 'watchers', 'assignee_id', 'reporter_id']

=== Missing/Zero Summary ===


,null_count,zero_count,non_null_count,total_rows,null_%,zero_%
assignee_id,10591,0,53398,63989,16.551282,0.000000
resolution,8039,0,55950,63989,12.563097,0.000000
resolved,8039,0,55950,63989,12.563097,0.000000
description,5929,0,58060,63989,9.265655,0.000000
priority,2655,0,61334,63989,4.149151,0.000000
reporter_id,2,0,63987,63989,0.003126,0.000000
project_name,1,0,63988,63989,0.001563,0.000000
repositoryname,1,0,63988,63989,0.001563,0.000000
project,1,0,63988,63989,0.001563,0.000000
key,1,0,63988,63989,0.001563,0.000000


In [ ]:
# ====== CLEANING CELL ======

# Copy the dataframe to avoid modifying the original
clean_df = df.copy()

# Fill text-based columns
text_cols = ["description", "resolution", "priority", "status", "title", "type", "project_name"]
for col in text_cols:
    if col in clean_df.columns:
        clean_df[col] = clean_df[col].fillna("Unknown").astype(str).str.strip().replace("", "Unknown")

# Fill numeric columns (votes, watchers, etc.)
num_cols = ["votes", "watchers"]
for col in num_cols:
    if col in clean_df.columns:
        clean_df[col] = clean_df[col].fillna(0).replace("", 0)

# Handle missing IDs or timestamps
if "assignee_id" in clean_df.columns:
    clean_df["assignee_id"] = clean_df["assignee_id"].fillna(-1).astype(int)
if "reporter_id" in clean_df.columns:
    clean_df["reporter_id"] = clean_df["reporter_id"].fillna(-1).astype(int)
if "resolved" in clean_df.columns:
    clean_df["resolved"] = pd.to_datetime(clean_df["resolved"], errors="coerce")
if "created" in clean_df.columns:
    clean_df["created"] = pd.to_datetime(clean_df["created"], errors="coerce")
if "updated" in clean_df.columns:
    clean_df["updated"] = pd.to_datetime(clean_df["updated"], errors="coerce")

# Fill remaining NaNs with "Unknown" or 0 as fallback
clean_df = clean_df.fillna({"description": "No description provided"}).fillna("Unknown")

# Confirm no missing left
print("\nAfter cleaning, null values per column:")
print(clean_df.isnull().sum())

# Quick check of cleaned rows
display(clean_df.head(5))



After cleaning, null values per column:
id                0
created           0
description       0
key               0
priority          0
project           0
project_name      0
repositoryname    0
resolution        0
resolved          0
status            0
title             0
type              0
updated           0
votes             0
watchers          0
assignee_id       0
reporter_id       0
dtype: int64


,id,created,description,key,priority,project,project_name,repositoryname,resolution,resolved,status,title,type,updated,votes,watchers,assignee_id,reporter_id
0,509981,2006-09-08 17:41:16.982000,A way to generate ebj/web projects with XDocle...,MECLIPSE-157,Major,MECLIPSE,Maven Eclipse Plugin,CODEHAUS,Duplicate,2007-03-14 06:36:43.842000,Closed,xdoclet facet for xdoclet-enabled ejb/web proj...,Improvement,2007-03-14 06:36:43.836000,0.0,1.0,9813,86730
1,509942,Unknown,"When a repository starts up, we attempt to reg...",MODE-1966,Major,MODE,ModeShape,JBOSS,Done,Unknown,Closed,Registering a repository in a read-only jndi c...,Bug,Unknown,0.0,2.0,553,553
2,509943,Unknown,Section 3.5.7.2 of the JSR-283 specification (...,MODE-1965,Critical,MODE,ModeShape,JBOSS,Done,Unknown,Closed,mixin types should not be allowed to inherit n...,Bug,Unknown,0.0,2.0,553,553
3,509944,Unknown,When property has been removed from node and t...,MODE-1964,Major,MODE,ModeShape,JBOSS,Done,Unknown,Closed,Connectors don't get removed properties in cha...,Bug,Unknown,0.0,3.0,553,86622
4,509982,2006-09-08 16:10:13.977000,The plugin should support setting Eclipse's te...,MECLIPSE-156,Major,MECLIPSE,Maven Eclipse Plugin,CODEHAUS,Fixed,2009-10-15 16:26:55.851000,Closed,Plugin should support setting file encoding,New Feature,2010-04-30 11:30:30.533000,6.0,4.0,16391,75580


In [ ]:
import re, html

def clean_text(text):
    if not isinstance(text, str):
        return ""
    text = html.unescape(text)  # decode HTML entities
    text = re.sub(r"<[^>]+>", " ", text)  # remove HTML tags
    text = re.sub(r"`{1,3}[^`]*`{1,3}", " ", text)  # remove code blocks
    text = re.sub(r"https?://\S+|www\.\S+", " ", text)  # remove URLs
    text = re.sub(r"[A-Za-z0-9._%+-]+@[A-Za-z0-9.-]+\.[A-Za-z]{2,}", " ", text)  # remove emails
    text = re.sub(r"\s+", " ", text)  # normalize whitespace
    text = re.sub(r"[^A-Za-z0-9 .,!?;:/()-]", " ", text)  # keep safe characters
    return text.strip().lower()

# Clean key text columns
for col in ["title", "description", "project_name", "type", "status", "priority"]:
    if col in clean_df.columns:
        clean_df[col] = clean_df[col].apply(clean_text)

# Combine into a single text field for embedding
clean_df["text_for_rag"] = (
    clean_df["title"].fillna("") + ". " +
    clean_df["description"].fillna("") + ". " +
    "Project: " + clean_df["project_name"].fillna("") + ". " +
    "Type: " + clean_df["type"].fillna("") + ". " +
    "Status: " + clean_df["status"].fillna("") + ". " +
    "Priority: " + clean_df["priority"].fillna("") + "."
)

# Preview results
display(clean_df[["id", "text_for_rag"]].head(5))


,id,text_for_rag
0,509981,xdoclet facet for xdoclet-enabled ejb/web proj...
1,509942,registering a repository in a read-only jndi c...
2,509943,mixin types should not be allowed to inherit n...
3,509944,connectors don t get removed properties in cha...
4,509982,plugin should support setting file encoding. t...


In [ ]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [ ]:
import json, os
from google.colab import files

# Create folder for cleaned outputs
os.makedirs("/content/clean_data", exist_ok=True)

# 1️⃣ Save as cleaned CSV
clean_csv_path = "/content/clean_data/jira_issues_clean.csv"
clean_df.to_csv(clean_csv_path, index=False)
print(f"✅ Cleaned CSV saved to: {clean_csv_path}")

# 2️⃣ Save as RAG JSONL (for vector DBs)
rag_jsonl_path = "/content/clean_data/jira_issues_rag.jsonl"

with open(rag_jsonl_path, "w", encoding="utf-8") as f:
    for _, row in clean_df.iterrows():
        doc = {
            "id": str(row["id"]),
            "text": row["text_for_rag"],
            "metadata": {
                "project": row.get("project_name", ""),
                "type": row.get("type", ""),
                "status": row.get("status", ""),
                "priority": row.get("priority", ""),
                "created": str(row.get("created", "")),
                "resolved": str(row.get("resolved", "")),
            },
        }
        f.write(json.dumps(doc, ensure_ascii=False) + "\n")

print(f"✅ JSONL for RAG saved to: {rag_jsonl_path}")

# 3️⃣ Show download buttons in Colab
print("\n⬇️ Click below to download your cleaned files:")
files.download(clean_csv_path)
files.download(rag_jsonl_path)


✅ Cleaned CSV saved to: /content/clean_data/jira_issues_clean.csv
✅ JSONL for RAG saved to: /content/clean_data/jira_issues_rag.jsonl

⬇️ Click below to download your cleaned files:


<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>